In [1]:
# clone project

!git clone https://github.com/unitaryai/detoxify


# install project

!pip install -e detoxify
%cd detoxify

# for training
!pip install -r requirements.txt

Cloning into 'detoxify'...
remote: Enumerating objects: 915, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 915 (delta 69), reused 56 (delta 39), pack-reused 787
Receiving objects: 100% (915/915), 52.02 MiB | 16.70 MiB/s, done.
Resolving deltas: 100% (516/516), done.
Obtaining file:///content/detoxify
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.1 MB/s eta 0:00:00
  Building editable for detoxify (pyproject.toml) ... done
  Created wheel for detoxify: filename=detoxify-0.5.1-0.editable-py3-none-any.whl size=11447 sha256=30f55a1d75b5c3a4f66f24

In [2]:
from google.colab import files

files.upload()

Saving jigsaw-toxic-comment-classification-challenge.zip to jigsaw-toxic-comment-classification-challenge.zip


In [ ]:
!mkdir jigsaw_data
!mkdir jigsaw_data/jigsaw-toxic-comment-classification-challenge
!unzip jigsaw-toxic-comment-classification-challenge.zip -d jigsaw_data
!unzip jigsaw_data/test.csv.zip -d jigsaw_data/jigsaw-toxic-comment-classification-challenge
!unzip jigsaw_data/train.csv.zip -d jigsaw_data/jigsaw-toxic-comment-classification-challenge
!unzip jigsaw_data/test_labels.csv.zip -d jigsaw_data/jigsaw-toxic-comment-classification-challenge

In [ ]:
#!python evaluate.py --checkpoint saved/lightning_logs/checkpoints/example_checkpoint.pth --test_csv test.csv


# DATA PERMUTATION

In [ ]:
import pandas as pd
import numpy as np
import re
import random

In [ ]:
def transform_text(text, method):
    if method == 'vowel_addition':
        return re.sub(r'([aeiou])', r'\1a', text)
    elif method == 'vowel_deletion':
        return re.sub(r'([aeiou])', '', text)
    elif method == 'vowel_substitution':
        vowels = 'aeiou'
        return re.sub(r'([aeiou])', lambda x: random.choice(vowels), text)
    elif method == 'leet_speak':
        leet_dict = {'a': '4', 'e': '3', 'i': '1', 'o': '0', 's': '5'}
        return ''.join(leet_dict.get(c, c) for c in text.lower())
    elif method == 'character_swap':
        chars = list(text)
        random.shuffle(chars)
        return ''.join(chars)
    else:
        return text

In [ ]:
def apply_random_adversarial_transformation(comment):
    # List of transformation methods
    transformation_methods = ['vowel_addition', 'vowel_deletion', 'vowel_substitution', 'leet_speak', 'character_swap']

    # Randomly choose one method
    chosen_method = random.choice(transformation_methods)

    # Apply the chosen method
    return transform_text(comment, chosen_method)

In [ ]:
swear_words_list = [
    "dogshit", "bullshit", '2g1c', '2 girls 1 cup', 'acrotomophilia', 'alabama hot pocket', 'alaskan pipeline', 'anal', 'anilingus', 'anus',
    'apeshit', 'arsehole', 'ass', 'asshole', 'assmunch', 'auto erotic', 'autoerotic', 'babeland', 'baby batter',
    'baby juice', 'ball gag', 'ball gravy', 'ball kicking', 'ball licking', 'ball sack', 'ball sucking', 'bangbros',
    'bareback', 'barely legal', 'barenaked', 'bastard', 'bastardo', 'bastinado', 'bbw', 'bdsm', 'beaner', 'beaners',
    'beaver cleaver', 'beaver lips', 'bestiality', 'big black', 'big breasts', 'big knockers', 'big tits', 'bimbos',
    'birdlock', 'bitch', 'bitches', 'black cock', 'blonde action', 'blonde on blonde action', 'blowjob', 'blow job',
    'blow your load', 'blue waffle', 'blumpkin', 'bollocks', 'bondage', 'boner', 'boob', 'boobs', 'booty call',
    'brown showers', 'brunette action', 'bukkake', 'bulldyke', 'bullet vibe', 'bullshit', 'bung hole', 'bunghole',
    'busty', 'butt', 'buttcheeks', 'butthole', 'camel toe', 'camgirl', 'camslut', 'camwhore', 'carpet muncher',
    'carpetmuncher', 'chocolate rosebuds', 'circlejerk', 'cleveland steamer', 'clit', 'clitoris', 'clover clamps',
    'clusterfuck', 'cock', 'cocks', 'coprolagnia', 'coprophilia', 'cornhole', 'coon', 'coons', 'creampie', 'cum',
    'cumming', 'cunnilingus', 'cunt', 'darkie', 'date rape', 'daterape', 'deep throat', 'deepthroat', 'dendrophilia',
    'dick', 'dildo', 'dingleberry', 'dingleberries', 'dirty pillows', 'dirty sanchez', 'doggie style', 'doggiestyle',
    'doggy style', 'doggystyle', 'dog style', 'dolcett', 'domination', 'dominatrix', 'dommes', 'donkey punch',
    'double dong', 'double penetration', 'dp action', 'dry hump', 'dvda', 'eat my ass', 'ecchi', 'ejaculation',
    'erotic', 'erotism', 'escort', 'eunuch', 'faggot', 'fecal', 'felch', 'fellatio', 'feltch', 'female squirting',
    'femdom', 'figging', 'fingerbang', 'fingering', 'fisting', 'foot fetish', 'footjob', 'frotting', 'fuck', 'fuck buttons',
    'fuckin', 'fucking', 'fucktards', 'fudge packer', 'fudgepacker', 'futanari', 'gang bang', 'gay sex', 'genitals',
    'giant cock', 'girl on', 'girl on top', 'girls gone wild', 'goatcx', 'goatse', 'god damn', 'gokkun', 'golden shower',
    'goodpoop', 'goo girl', 'goregasm', 'grope', 'group sex', 'g-spot', 'guro', 'hand job', 'handjob', 'hard core', 'hardcore',
    'hentai', 'homoerotic', 'honkey', 'hooker', 'hot carl', 'hot chick', 'how to kill', 'how to murder', 'huge fat', 'humping',
    'incest', 'intercourse', 'jack off', 'jail bait', 'jailbait', 'jelly donut', 'jerk off', 'jigaboo', 'jiggaboo', 'jiggerboo',
    'jizz', 'juggs', 'kike', 'kinbaku', 'kinkster', 'kinky', 'knobbing', 'leather restraint', 'leather straight jacket',
    'lemon party', 'lolita', 'lovemaking', 'make me come', 'male squirting', 'masturbate', "menage a trois", "milf", "missionary position", "motherfucker", "fucker", "fucking", "FUCK", "mound of venus", "mr hands",
    "muff diver", "muffdiving", "nambla", "nawashi", "negro", "neonazi", "nigga", "nigger", "nig nog",
    "nimphomania", "nipple", "nipples", "nsfw images", "nude", "nudity", "nympho", "nymphomania", "octopussy",
    "omorashi", "one cup two girls", "one guy one jar", "orgasm", "orgy", "paedophile", "paki", "panties", "panty",
    "pedobear", "pedophile", "pegging", "penis", "phone sex", "piece of shit", "pissing", "piss pig", "pisspig",
    "playboy", "pleasure chest", "pole smoker", "ponyplay", "poof", "poon", "poontang", "punany", "poop chute",
    "poopchute", "porn", "porno", "pornography", "prince albert piercing", "pthc", "pubes", "pussy", "queaf",
    "queef", "quim", "raghead", "raging boner", "rape", "raping", "rapist", "rectum", "reverse cowgirl", "rimjob",
    "rimming", "rosy palm", "rosy palm and her 5 sisters", "rusty trombone", "sadism", "santorum", "scat", "schlong",
    "scissoring", "semen", "sex", "sexo", "sexy", "shaved beaver", "shaved pussy", "shemale", "shibari", "shit",
    "shitblimp", "shitty", "shota", "shrimping", "skeet", "slanteye", "slut", "s&m", "smut", "snatch", "snowballing",
    "sodomize", "sodomy", "spic", "splooge", "splooge moose", "spooge", "spread legs", "spunk", "strap on", "strapon",
    "strappado", "strip club", "style doggy", "suck", "sucks", "suicide girls", "sultry women", "swastika", "swinger",
    "tainted love", "taste my", "tea bagging", "threesome", "throating", "tied up", "tight white", "tit", "tits",
    "titties", "titty", "tongue in a", "topless", "tosser", "towelhead", "tranny", "tribadism", "tub girl", "tubgirl",
    "tushy", "twat", "twink", "twinkie", "two girls one cup", "undressing", "upskirt", "urethra play", "urophilia",
    "vagina", "venus mound", "vibrator", "violet wand", "vorarephilia", "voyeur", "vulva", "wank", "wetback",
    "wet dream", "white power", "wrapping men", "wrinkled starfish", "xx", "xxx", "yaoi", "yellow showers",
    "yiffy", "zoophilia"
]

In [ ]:
def apply_adversarial_transformations(df):
    transformed_comments = []
    for index, row in df.iterrows():
        comment = row['comment_text']

        # Find all swear words using a simple pattern (you may need to enhance this pattern)
        swear_pattern = r'\b(' + '|'.join(re.escape(word) for word in swear_words_list) + r')\b'
        swear_words = re.findall(swear_pattern, comment, flags=re.IGNORECASE)
        # swear_words = re.findall(r'\b(bitch|bullshit|fuck)\b', comment, flags=re.IGNORECASE)

        # Apply a random transformation to each detected swear word
        for swear_word in swear_words:
            comment = comment.replace(swear_word, apply_random_adversarial_transformation(swear_word))

        transformed_comments.append(comment)
    df['transformed_comment'] = transformed_comments
    return df


In [29]:
y = pd.read_csv("jigsaw_data/jigsaw-toxic-comment-classification-challenge/test_updated.csv")

In [30]:
y

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,-1,-1,-1,-1,-1,-1
1,1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,-1,-1,-1,-1,-1,-1
2,2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",-1,-1,-1,-1,-1,-1
3,3,00017563c3f7919a,":If you have a look back at the source, the in...",-1,-1,-1,-1,-1,-1
4,4,00017695ad8997eb,I don't anonymously edit articles at all.,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...
153159,153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu...",-1,-1,-1,-1,-1,-1
153160,153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...,-1,-1,-1,-1,-1,-1
153161,153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ...",-1,-1,-1,-1,-1,-1
153162,153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the...",-1,-1,-1,-1,-1,-1


In [ ]:
transformation_methods = ['vowel_addition', 'vowel_deletion', 'vowel_substitution', 'leet_speak', 'character_swap']

In [13]:
train_df = pd.read_csv("jigsaw_data/jigsaw-toxic-comment-classification-challenge/train.csv")
adversarial_df = apply_adversarial_transformations(train_df)

In [14]:
adversarial_df = adversarial_df[['id', 'transformed_comment']]
print(adversarial_df)

                      id                                transformed_comment
0       0000997932d777bf  Explanation\nWhy the edits made under my usern...
1       000103f0d9cfb60f  D'aww! He matches this background colour I'm s...
2       000113f07ec002fd  Hey man, I'm really not trying to edit war. It...
3       0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...
4       0001d958c54c6e35  You, sir, are my hero. Any chance you remember...
...                  ...                                                ...
159566  ffe987279560d7ff  ":::::And for the second time of asking, when ...
159567  ffea4adeee384e90  You should be ashamed of yourself \n\nThat is ...
159568  ffee36eab5c267c9  Spitzer \n\nUmm, theres no actual article for ...
159569  fff125370e4aaaf3  And it looks like it was actually you who put ...
159570  fff46fc426af1f9a  "\nAnd ... I really don't think you understand...

[159571 rows x 2 columns]


In [17]:
test_df = pd.read_csv("jigsaw_data/jigsaw-toxic-comment-classification-challenge/test.csv")
adversarial_test_df = apply_adversarial_transformations(test_df)
adversarial_test_df = adversarial_test_df[['id', 'transformed_comment']]
print(adversarial_test_df)

                      id                                transformed_comment
0       00001cee341fdb12  Yo biatch Ja Rule is more succesful then you'l...
1       0000247867823ef7  == From RfC == \n\n The title is fine as it is...
2       00013b17ad220c46  " \n\n == Sources == \n\n * Zawe Ashton on Lap...
3       00017563c3f7919a  :If you have a look back at the source, the in...
4       00017695ad8997eb          I don't anonymously edit articles at all.
...                  ...                                                ...
153159  fffcd0960ee309b5  . \n i totally agree, this stuff is nothing bu...
153160  fffd7a9a6eb32c16  == Throw from out field to home plate. == \n\n...
153161  fffda9e8d6fafa9e  " \n\n == Okinotorishima categories == \n\n I ...
153162  fffe8f1340a79fc2  " \n\n == ""One of the founding nations of the...
153163  ffffce3fb183ee80  " \n :::Stop already. Your buallshiat is not w...

[153164 rows x 2 columns]


In [18]:
adversarial_df.to_csv('jigsaw_data/jigsaw-toxic-comment-classification-challenge/adversarial_dataset.csv', index=False)
adversarial_test_df.to_csv('jigsaw_data/jigsaw-toxic-comment-classification-challenge/adversarial_test_dataset.csv', index=False)

In [ ]:
# from google.colab import files
# files.download('adversarial_dataset.csv')

# BASELINE MODEL TRAINING

In [25]:
!python preprocessing_utils.py --test_csv jigsaw_data/jigsaw-toxic-comment-classification-challenge/test.csv --update_test --val_csv jigsaw_data/jigsaw-toxic-comment-classification-challenge/test_updated.csv  --create_val_set

!mv val.csv jigsaw_data/jigsaw-toxic-comment-classification-challenge

In [ ]:
!python train.py --config configs/Toxic_comment_classification_BERT.json --device 1

Global seed set to 1234
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
config.json: 100% 570/570 [00:00<00:00, 3.07MB/s]
model.safetensors: 100% 440M/440M [00:01<00:00, 432MB/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expec

#BASELINE MODEL EVAL ON ORIGINAL DATA

In [32]:
!cp model_eval/evaluate.py /content/detoxify

!python evaluate.py --checkpoint checkpoints/epoch\=3-step\=21276.ckpt --config configs/Toxic_comment_classification_BERT.json --test_csv jigsaw_data/jigsaw-toxic-comment-classification-challenge/test_updated.csv

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

#BASELINE MODEL EVAL ON TRANSFORMED DATA

In [38]:
temp = pd.read_csv("jigsaw_data/jigsaw-toxic-comment-classification-challenge/test_labels.csv")
temp.to_csv('jigsaw_data/jigsaw-toxic-comment-classification-challenge/adversarial_test_dataset_labels.csv', index=False)


!python preprocessing_utils.py --test_csv jigsaw_data/jigsaw-toxic-comment-classification-challenge/adversarial_test_dataset.csv --update_test --val_csv jigsaw_data/jigsaw-toxic-comment-classification-challenge/adversarial_test_dataset_updated.csv  --create_val_set

mv: cannot stat 'adversarial_test_dataset_updated.csv': No such file or directory


In [43]:
y = pd.read_csv("jigsaw_data/jigsaw-toxic-comment-classification-challenge/adversarial_test_dataset_updated.csv")
y= y.rename(columns={"transformed_comment": "comment_text"})
y.to_csv('jigsaw_data/jigsaw-toxic-comment-classification-challenge/adversarial_test_dataset_updated.csv', index=False)
y

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,00001cee341fdb12,Yo biatch Ja Rule is more succesful then you'l...,-1,-1,-1,-1,-1,-1
1,1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,-1,-1,-1,-1,-1,-1
2,2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",-1,-1,-1,-1,-1,-1
3,3,00017563c3f7919a,":If you have a look back at the source, the in...",-1,-1,-1,-1,-1,-1
4,4,00017695ad8997eb,I don't anonymously edit articles at all.,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...
153159,153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu...",-1,-1,-1,-1,-1,-1
153160,153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...,-1,-1,-1,-1,-1,-1
153161,153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ...",-1,-1,-1,-1,-1,-1
153162,153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the...",-1,-1,-1,-1,-1,-1


In [44]:
!python evaluate.py --checkpoint checkpoints/epoch\=3-step\=21276.ckpt --config configs/Toxic_comment_classification_BERT.json --test_csv jigsaw_data/jigsaw-toxic-comment-classification-challenge/adversarial_test_dataset_updated.csv

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i